In [ ]:
import pandas as pd, numpy as np, math, os
 import pandas as pd
import numpy as np
import snf
 # Load data
cytokines = pd.read_csv("/Users/SJp/Documents/project_local/VIB-LLM-SS/ml-summerschool-2025/PatStrat/data/Cytokines.csv", header=0, index_col=0).transpose()
clinical = pd.read_csv("/Users/SJp/Documents/project_local/VIB-LLM-SS/ml-summerschool-2025/PatStrat/data/Clinical.csv", header=0, index_col=0).transpose()
proteomics = pd.read_csv("/Users/SJp/Documents/project_local/VIB-LLM-SS/ml-summerschool-2025/PatStrat/data/MyelomaCells_proteomics.csv", header=0, index_col=0).transpose()
drugresponse = pd.read_csv("/Users/SJp/Documents/project_local/VIB-LLM-SS/ml-summerschool-2025/PatStrat/data/DrugResponse.csv", header=0, index_col=0).transpose().dropna()
 
# make sure all the above dataframes have the same patients in the same order
common_patients = set(clinical.index) & set(cytokines.index) & set(proteomics.index) & set(drugresponse.index)
common_patients = sorted(list(common_patients))
clinical = clinical.loc[common_patients].sort_index()
cytokines = cytokines.loc[common_patients].sort_index()
proteomics = proteomics.loc[common_patients].sort_index()
drugresponse = drugresponse.loc[common_patients].sort_index()

In [ ]:
# Load and preprocess data (assume clinical, cytokines, proteomics are already loaded and indexed by patient)
# Fill missing values with 0 for SNF
cytokines_data = cytokines.fillna(0).values
proteomics_data = proteomics.fillna(0).values
clinical_data = clinical.fillna(0).values

# Build similarity networks for each omics type
cytokines_aff = snf.make_affinity(cytokines_data, K=20, mu=0.5)
proteomics_aff = snf.make_affinity(proteomics_data, K=20, mu=0.5)

# Convert clinical categorical columns to one-hot encoding
clinical_numeric = pd.get_dummies(clinical)

# Fill missing values with 0
clinical_data = clinical_numeric.fillna(0).values

# Now build affinity matrix as before
clinical_aff = snf.make_affinity(clinical_data, K=20, mu=0.5)


/Users/SJp/Documents/project_local/VIB-LLM-SS/.venv/lib/python3.13/site-packages/sklearn/utils/deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/Users/SJp/Documents/project_local/VIB-LLM-SS/.venv/lib/python3.13/site-packages/sklearn/utils/deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/Users/SJp/Documents/project_local/VIB-LLM-SS/.venv/lib/python3.13/site-packages/sklearn/utils/deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/Users/SJp/Documents/project_local/VIB-LLM-SS/.venv/lib/python3.13/site-packages/sklearn/utils/deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/Users/SJp/Documents/project_local/VIB-LLM-SS/.venv/lib/

In [3]:
# Find common patients and align all dataframes
common_patients = set(clinical.index) & set(cytokines.index) & set(proteomics.index)
common_patients = sorted(list(common_patients))
clinical = clinical.loc[common_patients]
cytokines = cytokines.loc[common_patients]
proteomics = proteomics.loc[common_patients]

# Convert clinical categorical columns to one-hot encoding
clinical_numeric = pd.get_dummies(clinical)
clinical_data = clinical_numeric.fillna(0).values
cytokines_data = cytokines.fillna(0).values
proteomics_data = proteomics.fillna(0).values

# Build similarity networks for each omics type
import snf
cytokines_aff = snf.make_affinity(cytokines_data, K=20, mu=0.5)
proteomics_aff = snf.make_affinity(proteomics_data, K=20, mu=0.5)
clinical_aff = snf.make_affinity(clinical_data, K=20, mu=0.5)

# Integrate networks using SNF
networks = [cytokines_aff, proteomics_aff, clinical_aff]
fused_network = snf.snf(networks, K=20)

# Convert fused network to DataFrame
patients = clinical.index
fused_df = pd.DataFrame(fused_network, index=patients, columns=patients)

# Save fused similarity matrix
fused_df.to_csv("patient_snf_similarity.csv")

print("SNF patient similarity matrix saved as patient_snf_similarity.csv")

SNF patient similarity matrix saved as patient_snf_similarity.csv


/Users/SJp/Documents/project_local/VIB-LLM-SS/.venv/lib/python3.13/site-packages/sklearn/utils/deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/Users/SJp/Documents/project_local/VIB-LLM-SS/.venv/lib/python3.13/site-packages/sklearn/utils/deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/Users/SJp/Documents/project_local/VIB-LLM-SS/.venv/lib/python3.13/site-packages/sklearn/utils/deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/Users/SJp/Documents/project_local/VIB-LLM-SS/.venv/lib/python3.13/site-packages/sklearn/utils/deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/Users/SJp/Documents/project_local/VIB-LLM-SS/.venv/lib/

In [4]:
import re
import networkx as nx

def sanitize_keys(d):
    # Replace non-alphanumeric characters with underscores and remove leading underscores
    def clean_key(k):
        k = re.sub(r'[^0-9a-zA-Z_]', '_', str(k))
        k = re.sub(r'^_+', '', k)  # Remove leading underscores
        if not k or not k[0].isalpha():
            k = 'X' + k  # Ensure key starts with a letter
        return k
    return {clean_key(k): v for k, v in d.items()}

# Build graph from similarity matrix (e.g., keep top 3 edges per patient)
G = nx.Graph()
for pid in fused_df.index:
    # Add node with sanitized clinical metadata
    meta = sanitize_keys(clinical.loc[pid].to_dict())
    G.add_node(pid, **meta)
    # Add edges to top 3 similar patients
    sims = fused_df.loc[pid].drop(pid)
    top_neighbors = sims.sort_values(ascending=False).head(3)
    for neighbor_pid, score in top_neighbors.items():
        G.add_edge(pid, neighbor_pid, similarity=round(score, 4))

# Save to GML
nx.write_gml(G, "patient_similarity_with_metadata.gml")
print("Saved patient similarity network with clinical metadata to patient_similarity_with_metadata.gml")

Saved patient similarity network with clinical metadata to patient_similarity_with_metadata.gml


In [5]:
def class_from_val(v, pos_thr=0.15, neg_thr=-0.15):
    if v >= pos_thr:
        return "positive_effect"
    elif v <= neg_thr:
        return "adverse_effect"
    else:
        return "no_effect"
    
#for each column in drugresponse, apply class_from_val function  
drug_classes = drugresponse.applymap(class_from_val)
drug_classes.to_csv("/Users/SJp/Documents/project_local/VIB-LLM-SS/ml-summerschool-2025/PatStrat/data/drugresponse_classes.csv")

/var/folders/p3/70lzd1qn54d5cg3jhwm5szkm0000gp/T/ipykernel_21803/998341066.py:10: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  drug_classes = drugresponse.applymap(class_from_val)


In [ ]:
import importlib
import src.VIB_project_functions as FF
importlib.reload(FF)
results, patient_explanations=FF.run_drug_classification(proteomics, cytokines, clinical_numeric, drug_classes)

Skipping Bendamustine_DNA-alkylating agent (class counts: Counter({np.int64(1): 35, np.int64(0): 4}))
Skipping Bortezomib_Proteasome Inhibitor regime (class counts: Counter({np.int64(0): 21, np.int64(1): 18}))
Skipping Carfilzomib_Proteasome Inhibitor regime (class counts: Counter({np.int64(2): 22, np.int64(1): 16, np.int64(0): 1}))
Skipping Cisplatin_Platinum-containing (class counts: Counter({np.int64(1): 30, np.int64(0): 8, np.int64(2): 1}))
Skipping Cyclophosphamide_DNA-alkylating agent (class counts: Counter({np.int64(1): 35, np.int64(0): 3, np.int64(2): 1}))
Skipping Cytarabine_Nucleoitide analog (class counts: Counter({np.int64(1): 30, np.int64(2): 8, np.int64(0): 1}))


/Users/SJp/Documents/project_local/VIB-LLM-SS/.venv/lib/python3.13/site-packages/xgboost/training.py:183: UserWarning: [10:44:24] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


Skipping Etoposide_Topoisomerase inhibitor (class counts: Counter({np.int64(1): 27, np.int64(2): 11, np.int64(0): 1}))


/Users/SJp/Documents/project_local/VIB-LLM-SS/.venv/lib/python3.13/site-packages/xgboost/training.py:183: UserWarning: [10:44:25] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/Users/SJp/Documents/project_local/VIB-LLM-SS/.venv/lib/python3.13/site-packages/xgboost/training.py:183: UserWarning: [10:44:26] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


Skipping Melphalan_DNA-alkylating agent (class counts: Counter({np.int64(1): 31, np.int64(0): 7, np.int64(2): 1}))


...

/Users/SJp/Documents/project_local/VIB-LLM-SS/.venv/lib/python3.13/site-packages/xgboost/training.py:183: UserWarning: [10:44:32] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


Skipping Dexamethasone+Bortezomib+Thalidomide_Proteasome Inhibitor regime (class counts: Counter({np.int64(2): 24, np.int64(1): 14, np.int64(0): 1}))
Skipping Dexamethasone+Carfilzomib+Cyclophosphamide_Proteasome Inhibitor regime (class counts: Counter({np.int64(1): 30, np.int64(0): 9}))
Skipping Dexamethasone+Carfilzomib+Lenalidomide_Proteasome Inhibitor regime (class counts: Counter({np.int64(1): 30, np.int64(0): 9}))


/Users/SJp/Documents/project_local/VIB-LLM-SS/.venv/lib/python3.13/site-packages/xgboost/training.py:183: UserWarning: [10:44:33] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


Skipping Dexamethasone+Carfilzomib+Pomalidomide_Proteasome Inhibitor regime (class counts: Counter({np.int64(1): 28, np.int64(0): 11}))
Skipping Dexamethasone+Ixazomib+Cyclophosphamide_Proteasome Inhibitor regime (class counts: Counter({np.int64(1): 24, np.int64(2): 14, np.int64(0): 1}))


/Users/SJp/Documents/project_local/VIB-LLM-SS/.venv/lib/python3.13/site-packages/xgboost/training.py:183: UserWarning: [10:44:34] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


In [7]:
# import json

# # Save patient explanations
# with open("patient_DRP_results.json", "w") as f:
#     json.dump(patient_explanations, f, indent=2)

# patient_explanations[0]

# for patient in patient_explanations:
#     with open(f"{patient}_DRP_results.json", "w") as f:
#     json.dump(patient_explanations, f, indent=2)

#save patient_explanations per patient to json
#shap_values
#results


In [ ]:
FF.save_patient_graphs_json(patient_explanations, results, clinical, outdir="/Users/SJp/Documents/project_local/VIB-LLM-SS/ml-summerschool-2025/topic-1_data-integration-and-llms/project/results/patient_graphs_json")

/Users/SJp/Documents/project_local/VIB-LLM-SS/.venv/lib/python3.13/site-packages/networkx/readwrite/json_graph/node_link.py:145: FutureWarning: 
The default value will be `edges="edges" in NetworkX 3.6.

To make this warning go away, explicitly set the edges kwarg, e.g.:

  nx.node_link_data(G, edges="links") to preserve current behavior, or
  nx.node_link_data(G, edges="edges") for forward compatibility.
  warnings.warn(
/Users/SJp/Documents/project_local/VIB-LLM-SS/.venv/lib/python3.13/site-packages/networkx/readwrite/json_graph/node_link.py:145: FutureWarning: 
The default value will be `edges="edges" in NetworkX 3.6.

To make this warning go away, explicitly set the edges kwarg, e.g.:

  nx.node_link_data(G, edges="links") to preserve current behavior, or
  nx.node_link_data(G, edges="edges") for forward compatibility.
  warnings.warn(
/Users/SJp/Documents/project_local/VIB-LLM-SS/.venv/lib/python3.13/site-packages/networkx/readwrite/json_graph/node_link.py:145: FutureWarning: 
The

Saved patient_graphs_json/MM058_KG.json
Saved patient_graphs_json/MM028_KG.json
Saved patient_graphs_json/MM072_KG.json
Saved patient_graphs_json/MM048_KG.json
Saved patient_graphs_json/MM036_KG.json
Saved patient_graphs_json/MM092_KG.json
Saved patient_graphs_json/MM042_KG.json
Saved patient_graphs_json/MM070_KG.json
Saved patient_graphs_json/MM044_KG.json
Saved patient_graphs_json/MM062_KG.json
Saved patient_graphs_json/MM106_KG.json
Saved patient_graphs_json/MM081_KG.json
Saved patient_graphs_json/MM084_KG.json
Saved patient_graphs_json/MM049_KG.json
Saved patient_graphs_json/MM103_KG.json
Saved patient_graphs_json/MM037_KG.json
Saved patient_graphs_json/MM039_KG.json
Saved patient_graphs_json/MM078_KG.json
Saved patient_graphs_json/MM059_KG.json
Saved patient_graphs_json/MM067_KG.json
Saved patient_graphs_json/MM082_KG.json
Saved patient_graphs_json/MM051_KG.json
Saved patient_graphs_json/MM046_KG.json
Saved patient_graphs_json/MM065_KG.json
Saved patient_graphs_json/MM026_KG.json


/Users/SJp/Documents/project_local/VIB-LLM-SS/.venv/lib/python3.13/site-packages/networkx/readwrite/json_graph/node_link.py:145: FutureWarning: 
The default value will be `edges="edges" in NetworkX 3.6.

To make this warning go away, explicitly set the edges kwarg, e.g.:

  nx.node_link_data(G, edges="links") to preserve current behavior, or
  nx.node_link_data(G, edges="edges") for forward compatibility.
  warnings.warn(


### GSEA 

In [42]:
import json
with open("patient_res_jsons/MM082.json", "r") as f:
    patient_data = json.load(f)

id = 'MM082'
drug_keys = list(patient_data[id]["Drugs"].keys())

In [30]:
pos_features_all = {}
neg_features_all = {}

for drug in drug_keys:
    pos_features = patient_data[id]["Drugs"][drug]['SHAP']['Top_Positive']
    neg_features = patient_data[id]["Drugs"][drug]['SHAP']['Top_Negative']
    
    # Clean positive features for this drug
    pos_features_all[drug] = [
        item["Feature"].split("_", 1)[1] if "_" in item["Feature"] else item["Feature"]
        for item in pos_features
    ]
    
    # Clean negative features for this drug
    neg_features_all[drug] = [
        item["Feature"].split("_", 1)[1] if "_" in item["Feature"] else item["Feature"]
        for item in neg_features
    ]



In [33]:
from typing import List
from langchain_core.tools import tool
from gseapy import enrichr

@tool
def enrichr_query(gene_list: List[str]):
    """Run enrichment analysis on a list of genes.

    This tool allows to run enrichment analysis on a list of genes using the `gseapy` library.
    Using this tool, a model can get information about the biological processes enriched in a set of genes.

    Args:
        gene_list: list of genes to run enrichment analysis on

    Returns:
        DataFrame: DataFrame containing the enrichment results
    """
    # Run enrichment
    enr = enrichr(
        gene_list=gene_list,
        gene_sets='GO_Biological_Process_2021',
        organism='Human',
        outdir=None,  # no files will be written
        cutoff=0.05
    )

    # Save results as DataFrame
    df_results = enr.results

    return df_results

In [34]:
from langchain.chat_models import init_chat_model
from langchain_core.prompts import ChatPromptTemplate

# Create the base model (configure your provider and model ID as needed)
llm = init_chat_model(model="gemini-2.5-flash", model_provider="google_genai")

# Bind the tool(s)
llm_with_tools = llm.bind_tools([enrichr_query])

# A simple prompt: system sets expectations; human supplies the input genes
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful bioinformatics assistant. Use tools when needed."),
    ("human", "{question}")
])

chain = prompt | llm_with_tools

In [41]:
from collections import defaultdict

# Step 1: Collect SHAP features per predicted class
class_features = defaultdict(lambda: {"positive": [], "negative": []})

for drug in drug_keys:
    predicted_class = patient_data[id]["Drugs"][drug]["Predicted_Class"]
    pos_features = patient_data[id]["Drugs"][drug]['SHAP']['Top_Positive']
    neg_features = patient_data[id]["Drugs"][drug]['SHAP']['Top_Negative']
    
    # Clean and add features
    class_features[predicted_class]["positive"].extend(
        [item["Feature"].split("_", 1)[1] if "_" in item["Feature"] else item["Feature"]
         for item in pos_features]
    )
    class_features[predicted_class]["negative"].extend(
        [item["Feature"].split("_", 1)[1] if "_" in item["Feature"] else item["Feature"]
         for item in neg_features]
    )

# Step 2: Run enrichment for each class
results_by_class = {}

for cls, feats in class_features.items():
    results_by_class[cls] = {}
    
    if feats["positive"]:
        results_by_class[cls]["positive"] = enrichr_query({"gene_list": list(set(feats["positive"]))})
    else:
        results_by_class[cls]["positive"] = None
    
    if feats["negative"]:
        results_by_class[cls]["negative"] = enrichr_query({"gene_list": list(set(feats["negative"]))})
    else:
        results_by_class[cls]["negative"] = None

# Step 3: Summarize per class
for cls, res in results_by_class.items():
    question = f"Predicted class: {cls}\n\nSummarize the functional biology or pathways SHAP features.\n"
    
    if res["positive"] is not None and not res["positive"].empty:
        question += f"\nPositive SHAP features (supporting {cls}):\n{res['positive'].head(10).to_string(index=False)}\n"
    
    if res["negative"] is not None and not res["negative"].empty:
        question += f"\nNegative SHAP features (against {cls}):\n{res['negative'].head(10).to_string(index=False)}\n"
    
    ai_msg = chain.invoke({"question": question})
    print(f"\n### Drugs Predicted as {cls}")
    print(ai_msg.content)



### Drugs Predicted as no_effect
The SHAP features indicate distinct biological processes influencing the "no_effect" prediction.

**Positive SHAP features** (supporting "no_effect") highlight pathways related to:
*   **Immune regulation**, specifically the positive regulation of T cell proliferation and activation, and lymphocyte proliferation (involving genes like IL15 and CD3E). This also includes general regulation of immune response.
*   **Cellular adhesion** mediated by integrins (e.g., CD3E, ADA).
*   **Posttranscriptional regulation of gene expression** (e.g., NUDT21, RBM4B).
*   **Protein-containing complex assembly** (e.g., CUL1, SNX9, CD3E).
*   Less prominently, substantia nigra development and cellular response to indole-3-methanol.

**Negative SHAP features** (arguing against "no_effect") point towards:
*   **Neutrophil-mediated immunity**, including degranulation and activation (involving genes like CTSZ, HLA-B, HP).
*   **T cell mediated cytotoxicity and immunity**, wi

### summarizing patient knowledge graph

In [ ]:
from langchain.chat_models import init_chat_model

# Pick via provider:model string → works across providers
llm = init_chat_model(model="gemini-2.5-flash",
                      model_provider="google_genai",
                      temperature=0.2)


llm.invoke("What is the capital of France?")

In [ ]:
# You already have:
# from langchain.chat_models import init_chat_model
# llm = init_chat_model(model="gemini-2.5-flash", model_provider="google_genai", temperature=0.2)
pkg_path = "patient_graphs_json/MM082_KG.json"   # your PKG file
patient_id = "MM082"

result_json, raw = ask_llm_compare_drugs(bundle, llm, return_json=True)

# Inspect
print(raw)  # full prose + JSON
if result_json:
    print(json.dumps(result_json, indent=2))


NameError: name 'build_drug_comparison_bundle' is not defined